In [1]:
import tensorflow as tf
import numpy as np

In [97]:
#Verify Tensorflow is using GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [98]:
lst = []

with open('../data/full_tab_string_test.txt') as small_pf:

    tmp_list = []
    for line in small_pf:
        line = line.rstrip("\n")
        if line == "":
            lst.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.extend(line.split())

    if tmp_list:  # add last one
        lst.append(tmp_list)
lst = lst[0]

In [99]:
text = ""
with open('../data/full_tab_string_test.txt') as small_pf:
    text = small_pf.read()

In [100]:
#Can't use set because tab encodings are not hashable types
def unique(list1):
    unique_set = set()
    for x in list1:
        if x not in unique_set:
            unique_set.add(x)
    return unique_set

In [101]:
#list of all tab locations
tab_vocab = unique(lst)

In [102]:
#this is a quick fix and needs to be done at the raw data level
new_list = []
for tab in tab_vocab:
    if int(tab[1:]) <= 22:
        new_list.append(tab)
tab_vocab = new_list

In [103]:
ids_from_chars = tf.keras.layers.StringLookup(
    num_oov_indices = 1, 
    vocabulary=list(tab_vocab), 
    mask_token=None)
print(ids_from_chars.get_vocabulary())

['[UNK]', 'G19', 'A4', 'D12', 'E6', 'e15', 'e17', 'e16', 'G10', 'e18', 'D0', 'G11', 'A15', 'G6', 'e0', 'D10', 'A1', 'D1', 'B8', 'B7', 'e12', 'E17', 'A2', 'B4', 'D6', 'G8', 'B13', 'D15', 'E9', 'B3', 'B9', 'B2', 'A14', 'A12', 'e8', 'B1', 'e14', 'E7', 'G15', 'E3', 'A11', 'E19', 'D17', 'D4', 'D14', 'D5', 'B12', 'B17', 'D9', 'B6', 'G16', 'e1', 'G9', 'e7', 'e2', 'A10', 'D11', 'B20', 'B10', 'B15', 'A5', 'A6', 'e11', 'B14', 'B18', 'G18', 'E14', 'G21', 'D7', 'B11', 'B0', 'e4', 'G4', 'E10', 'e5', 'e9', 'e21', 'A3', 'G17', 'G3', 'E11', 'D16', 'E8', 'B16', 'B19', 'B21', 'D2', 'E0', 'e6', 'G12', 'E4', 'e19', 'A7', 'e13', 'G2', 'A8', 'A9', 'D3', 'A0', 'G0', 'A13', 'e3', 'G1', 'G7', 'E12', 'D8', 'e20', 'D13', 'G5', 'G14', 'D19', 'B5', 'e10', 'E2', 'e22', 'E5', 'E1']


In [104]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [105]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids) + " ", axis=-1)

In [106]:
all_ids = ids_from_chars(tf.strings.split(text))

In [107]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [108]:
seq_length = 10

In [109]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print(type(sequences))
for seq in sequences.take(1):
  print(chars_from_ids(seq))
  print(type(seq))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
tf.Tensor([b'D7' b'G5' b'B5' b'e5' b'e7' b'D6' b'B5' b'G5' b'e7' b'e8' b'D5'], shape=(11,), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [110]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [111]:
dataset = sequences.map(split_input_target)
dataset



<MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [112]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 '
Target: b'G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '


In [113]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 10), dtype=tf.int64, name=None), TensorSpec(shape=(64, 10), dtype=tf.int64, name=None))>

In [114]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 1024

In [115]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    #adjust the dense units size                         
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [116]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [117]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 117) # (batch_size, sequence_length, vocab_size)


In [118]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  7488      
                                                                 
 gru (GRU)                   multiple                  3348480   
                                                                 
 dense (Dense)               multiple                  119925    
                                                                 
Total params: 3,475,893
Trainable params: 3,475,893
Non-trainable params: 0
_________________________________________________________________


In [119]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [120]:
sampled_indices

array([ 72,  29,  88,  81, 105,  74,  78,  40,  89,   2], dtype=int64)

In [121]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [122]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 10, 117)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7617264, shape=(), dtype=float32)


In [123]:
model.compile(optimizer='adam', loss=loss)

In [124]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './tab_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [125]:
EPOCHS = 100

In [126]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
27/27 [==============================] - 2s 14ms/step - loss: 4.2605
Epoch 2/100
27/27 [==============================] - 0s 13ms/step - loss: 3.6190
Epoch 3/100
27/27 [==============================] - 0s 11ms/step - loss: 3.2224
Epoch 4/100
27/27 [==============================] - 0s 11ms/step - loss: 2.9733
Epoch 5/100
27/27 [==============================] - 0s 10ms/step - loss: 2.7475
Epoch 6/100
27/27 [==============================] - 0s 11ms/step - loss: 2.5544
Epoch 7/100
27/27 [==============================] - 0s 12ms/step - loss: 2.4127
Epoch 8/100
27/27 [==============================] - 0s 10ms/step - loss: 2.2798
Epoch 9/100
27/27 [==============================] - 0s 10ms/step - loss: 2.1486
Epoch 10/100
27/27 [==============================] - 0s 11ms/step - loss: 2.0437
Epoch 11/100
27/27 [==============================] - 0s 11ms/step - loss: 1.9141
Epoch 12/100
27/27 [==============================] - 0s 10ms/step - loss: 1.7996
Epoch 13/100
27/27 [=====

In [127]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, omit, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.

    skip_ids = self.ids_from_chars(omit)[:, None]

    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, omit, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.

    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [128]:
from MidiToTabOptions import *
tab_midi_dict = generate_tab_midi_pairs()

In [129]:
from MidiToArray import *
notes = MidiToArray(r"C:\Users\alecr\desktop\mutab\Sandbox\samples\sample.mid")
notes = notes[1:16]
notes



[60, 64, 69, 71, 64, 60, 71, 72, 64, 60, 72, 66, 62, 57, 66]

In [130]:
#Array of Midi notes
#notes = [60, 61, 63, 60, 56, 58]
states = None

#First tab note, hard coded
base_tab = 'D7 '

#Dynamically starting input 
append_tab = tf.constant([base_tab])
for note in notes:

    #Finds all tab options for the current note and 
    #removes them from the omitted option list in the OneStep function
    options = ['[UNK]']
    options.extend(tab_midi_dict[note])
    omit = list(set(ids_from_chars.get_vocabulary()) - set(options))
    omit.insert(0, '[UNK]')

    omit_sorted = []
    for item in ids_from_chars.get_vocabulary():
        if item in omit:
            omit_sorted.append(item)

    
    #generates the next predicted tab note out of the set of possible options   
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars, omit_sorted)

    #bootstrap the predictions 
    bootstrap_list = []    
    for i in range(100):
        next_char, states = one_step_model.generate_one_step(append_tab, omit_sorted, states=states)    
        bootstrap_list.append(next_char.numpy()[0].decode('utf-8'))
    next_tab = max(set(bootstrap_list), key=bootstrap_list.count)


    #append new tab to input list
    append_tab = tf.constant([append_tab.numpy()[0].decode('utf-8') + next_tab + ' '])
tf.keras.backend.clear_session()

In [131]:
string = str(append_tab.numpy()[0].decode('utf-8'))
array = string.split(' ')[:-1]
print(convert_array_to_tab(array))

e|------------------------------------------------
B|------------12----1--12-------1-----------------
G|---5--------------------17-------17-------2-----
D|7-----14-19----14----------14-------16-12----16-
A|------------------------------------------------
E|------------------------------------------------



In [132]:
print(append_tab)

tf.Tensor([b'D7 G5 D14 D19 B12 D14 B1 B12 G17 D14 B1 G17 D16 D12 G2 D16 '], shape=(1,), dtype=string)
